In [4]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from spkit import pfa
from datetime import timedelta

ModuleNotFoundError: No module named 'statsmodels'

In [6]:
!pip install statsmodels

     |████████████████████████████████| 8.1MB 2.0MB/s eta 0:00:01
     |████████████████████████████████| 235kB 1.9MB/s eta 0:00:01


In [7]:
%load_ext autoreload

In [8]:
%autoreload 1

In [9]:
%aimport spkit.pfa

In [7]:
import random
for i in range(10):
    print(random.randint(1,2))

1
1
2
1
1
1
1
2
1
2


In [34]:
df = pd.read_csv("./data/kdd/dummy/data.csv")

import random

# Time decay per skill
def calc_time(row):
    # First time
    if not (row['wins'] + row['fails']):
        row['elapsed_previous'] = 0
    else:
        row['elapsed_previous'] = random.randint(1,2)
    return row

df = df.apply(calc_time, axis=1)
df

,user,item,skill,correct,wins,fails,elapsed_previous
0,1,1,1,1,0,0,0
1,1,2,2,0,0,0,0
2,2,1,1,0,0,0,0
3,2,1,1,1,0,1,1
4,2,2,2,0,0,0,0


In [35]:
ep = df['elapsed_previous'].unique()
ep_array = ep.reshape(-1, 1)
enc = OneHotEncoder(sparse=False, categories='auto')
ep_onehot = enc.fit_transform(ep_array)
ep_onehot

array([[1., 0.],
       [0., 1.]])

In [36]:
df = pd.concat((df, pd.DataFrame(enc.transform(pd.DataFrame(df['elapsed_previous'])), columns=["status 0", "status 1"])), axis=1)

In [37]:
df

,user,item,skill,correct,wins,fails,elapsed_previous,status 0,status 1
0,1,1,1,1,0,0,0,1.0,0.0
1,1,2,2,0,0,0,0,1.0,0.0
2,2,1,1,0,0,0,0,1.0,0.0
3,2,1,1,1,0,1,1,0.0,1.0
4,2,2,2,0,0,0,0,1.0,0.0


Onehot encoding wins and fails per skill

In [9]:
skills = df['skill'].unique()
skills_array = skills.reshape(-1, 1)
type(skills)

numpy.ndarray

In [10]:
enc = OneHotEncoder(sparse=False, categories='auto')
skills_onehot = enc.fit_transform(skills_array)
skills_onehot

array([[1., 0.],
       [0., 1.]])

In [20]:
pd.DataFrame(df['elapsed_previous'])

,elapsed_previous
0,0
1,0
2,0
3,2
4,0


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [11]:
skills_onehot[0]

array([1., 0.])

In [12]:
def create_onehot(row):
    idx = np.where(skills == row['skill'])[0]
    wins = row['wins']*skills_onehot[idx][0]
    print(wins)
    fails = row['fails']*skills_onehot[idx][0]
    return np.concatenate((skills_onehot[idx][0], wins, fails))

onehot_array = df.apply(create_onehot, axis=1)
cols = ["skills_%d"%skill for skill in skills]
cols += ["wins_%d"%skill for skill in skills]
cols += ["fails_%d"%skill for skill in skills]

[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]


In [13]:
df = pd.DataFrame(pfa_data, columns=["skill", "wins", "fails", "outcome"])

NameError: name 'pfa_data' is not defined

In [14]:
df

,user,item,skill,correct,wins,fails
0,1,1,1,1,0,0
1,1,2,2,0,0,0
2,2,1,1,0,0,0
3,2,1,1,1,0,1
4,2,2,2,0,0,0


In [15]:
onehot_array

0    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
2    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
3    [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
4    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
dtype: object

In [16]:
onehot_df = pd.DataFrame(onehot_array.tolist(), columns=cols)
df = pd.concat((df,onehot_df), axis=1)
df

,user,item,skill,correct,wins,fails,skills_1,skills_2,wins_1,wins_2,fails_1,fails_2
0,1,1,1,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,2,2,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0
2,2,1,1,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0
3,2,1,1,1,0,1,1.0,0.0,0.0,0.0,1.0,0.0
4,2,2,2,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0


In [17]:
X = df[cols]
X = sm.add_constant(X, prepend=False)
y = df['correct']

NameError: name 'sm' is not defined

In [3]:
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

In [4]:
model = LogisticRegression(fit_intercept=False, penalty='elasticnet', solver='saga', l1_ratio=0.01)
mdl = model.fit(X, y)
model.coef_

NameError: name 'X' is not defined

In [108]:
model.predict_proba(X)

array([[0.47851518, 0.52148482],
       [0.69981883, 0.30018117],
       [0.47851518, 0.52148482],
       [0.38570601, 0.61429399],
       [0.69981883, 0.30018117]])

In [109]:
model.predict(X)

array([1, 0, 1, 1, 0])

In [110]:
model.coef_[0]/res.params.values

/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([0.67294775, 0.10862896,        nan,        nan, 0.07167657,
              nan, 0.51080627])

In [111]:
model = sm.GLM(y, X, family=sm.families.Binomial())
res = model.fit_regularized(alpha=0.001)
res.params

skills_1    0.499466
skills_2   -5.489446
wins_1      0.000000
wins_2      0.000000
fails_1     5.293305
fails_2     0.000000
const      -0.489662
dtype: float64

In [112]:
predictions = res.predict(X)
predictions_nominal = [ 0 if x < 0.5 else 1 for x in predictions]
predictions_nominal

[1, 0, 1, 1, 0]

In [87]:
predictions

0    0.502451
1    0.002525
2    0.502451
3    0.995049
4    0.002525
dtype: float64

In [25]:
res.summary()

/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                correct   No. Observations:                    5
Model:                            GLM   Df Residuals:                        2
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1.3863
Date:                Tue, 15 Oct 2019   Deviance:                       2.7726
Time:                        17:07:29   Pearson chi2:                     2.00
No. Iterations:                    20                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
skills_1   -3.429e-24      1.414  -2.42e-24      1.000      -2.772       2.772
skills_2     -21.5661   2.07e+04     -0.001      0.999   -4.05e+04    4.05e+04
wins_1              0          0        nan        nan           0           0
wins_2              0          0        nan        nan           0           0
fails_1       21.5661   2.92e+04      0.001      0.999   -5.73e+04    5.73e+04
fails_2             0          0        nan        nan           0           0
==============================================================================
"""

In [45]:
q_matrix = np.asarray([[1,0], [0,1]])
data = [[1,1,0],[0,1,1],[0,2,0],[1, 2,0],[0,2,1]]
data = np.asarray(data)

In [46]:
q_matrix

array([[1, 0],
       [0, 1]])

In [47]:
data

array([[1, 1, 0],
       [0, 1, 1],
       [0, 2, 0],
       [1, 2, 0],
       [0, 2, 1]])

In [49]:
model = pfa.PFA(lib="sklearn")
model.fit(data, q_matrix, penalty="elasticnet", solver="saga", l1_ratio=0.01)

In [70]:
student_idx = np.where(data[:,1] == 2)
outcome = data[student_idx, 0].astype('int64')
question_id = data[student_idx, 2].astype('int64')
data_student = np.vstack((outcome, question_id)).T.astype('int64').tolist()
data_student[0:2]

[[0, 0], [1, 0]]

In [61]:
df = model._transform_student_data(data_student, q_matrix)
df[0]

,skill,wins,fails,outcome,skills_0,skills_1,wins_0,wins_1,fails_0,fails_1
0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0
1,0,0,1,1,1.0,0.0,0.0,0.0,1.0,0.0
2,1,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0


In [74]:
df = model._transform_student_data(data_student[2:], q_matrix)
df[0]

,skill,wins,fails,outcome,skills_0,skills_1,wins_0,wins_1,fails_0,fails_1
0,1,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0


In [35]:
model.params

{'skills': [0, 1], 'skills_onehot': array([[1., 0.],
        [0., 1.]]), 'weights': array([[-0.62252345,  0.48941992, -0.48983904,  0.        ,  0.        ,
          0.42237643,  0.        ]])}

In [52]:
pfa_onehot = model._transform_student_data(data_student, q_matrix)
pfa_onehot[0]

,skill,wins,fails,outcome,skills_0,skills_1,wins_0,wins_1,fails_0,fails_1
0,0,0,0,1,1.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0


In [53]:
model.predict(data_student, q_matrix)

array([0, 0], dtype=int64)

In [54]:
model.predict_proba(data_student, q_matrix)

array([[0.53324205, 0.46675795],
       [0.75258094, 0.24741906]])

In [95]:
model.predict_proba(data, q_matrix)

array([[0.53323083, 0.46676917],
       [0.75257296, 0.24742704],
       [0.53323083, 0.46676917],
       [0.4281841 , 0.5718159 ],
       [0.75257296, 0.24742704]])

In [96]:
model.params

array([[-0.62254475,  0.48942519, -0.48983743,  0.        ,  0.        ,
         0.42238341,  0.        ]])

In [97]:
y = np.asarray(data)[:,0]
y_pred_proba = model.predict_proba(data, q_matrix)
model.score(y, y_pred_proba)

(-2.5181741566466918,
 19.036348313293384,
 16.302413700332085,
 0.4019926570488578,
 0.8)

In [102]:
y

array([1, 0, 0, 1, 0])

In [98]:
model = pfa.PFA(lib="sm")
model.fit(data, q_matrix, penalty="elasticnet", L1_wt=0.01, alpha=0.2)
model.predict(data, q_matrix)

/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


array([1, 0, 1, 1, 0])

In [101]:
model.params

array([[-0.25012519,  0.33609478, -0.59632098,  0.        ,  0.        ,
         0.37948703,  0.        ]])

In [100]:
y = np.asarray(data)[:,0]
y_pred_proba = model.predict_proba(data, q_matrix)
model.score(y, y_pred_proba)

/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


(-2.5892616519252836,
 19.178523303850568,
 16.44458869088927,
 0.4074054197810027,
 0.8)